# Parser Evaluation

There is an [earlier version](https://colab.research.google.com/github/petervdonovan/CitationParser/blob/master/ParserEvaluation.ipynb) of this notebook, and much of the work contained in this notebook is different from but parallel to this earlier version. There are a few reasons why it was necessary to rethink this prior work:
1. Tags will not be a component of the testing or training dataset.
    * This does not mean that tags will not continue to be important. However, I have decided that the our objective -- use of citations to create edges in a social network -- does not inherently involve tagging.
1. In lieu of tags, metadata is being included in a format that is as independent from its origin as possible.
    * Names in particular do require this. In bibliographic entries, the formatting of names varies widely by style guide. If names are to be considered as semantic information instead of as raw text, they should be presented in a data structure that makes the different parts of a name explicit.
1. Metrics used to evaluate models will be carefully chosen to make them as interpretable as possible for individuals who do not know any low-level details of how our models operate.
    * Changes in this direction reduce the amount of text that is required alongside any reported metrics.

There may be several parts of this notebook where it is tempting to cry out, "I have seen all of this before, in another notebook! This is not DRY!" I insist on justifying myself by noting that in writing this notebook I have slightly different purposes in mind, and that I do not wish to be constrained by links or dependencies with an implementation that I may not wish to keep.

In [4]:
!pip install pickle5
import pandas as pd
import numpy as np
import pickle5 as pickle
import random
import time
import matplotlib.pyplot as plt
from gensim.utils import simple_preprocess
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Datasets

### Note on dataset selection

For the moment, I will be loading the OpenCitations dataset for initial tests. Much of the motivation behind our use of this dataset is replicability, although it does have a replicability issue of its own: Because the OpenCitations dataset is too large to support a SPARQL query that asks for the DOI of _every_ cited work for which we have a raw-text citation, I had to use a LIMIT clause. There are no promises about whether the sample I got was representative of the corpus, so this is a little unfortunate.

Another challenge is that the Zotero dataset does not have raw text citations that are reliably matched with the corresponding metadata. A _mostly_ successful attempt to tackle this issue is included in [this notebook](https://colab.research.google.com/drive/1OEFWVWgEzCiPA35Ma20svEY5OhPCuPwI), but it is a beast, and the output has imperfections of a severity that is difficult to quantify definitively.

In any case, the OpenCitations dataset will be used for development. Long-term goals might be:
* To include a more representative (or complete) sample of the OpenCitations corpus, for replicability, and
* To include the Zotero dataset.

### Fields in the Dataset

The dataset has the following fields for the models to use for prediction:
* **raw_text**: The raw text citation, represented as a string.

The dataset has the following fields for the models to predict:
* **author**: The name(s) of the authors of the work, represented as `Name` objects.
* **year**: The publication year of the work, represented as an integer.
* **title**: The title of the work, represented as a string.

These three fields are the ones that are likely candidates to be used to construct edges; the others are much more questionable. Each of them is commonly found -- for example, 98.8% of records in the Zotero database have a publication year, 99.6% have an author, and 99.7% have a title.

Every row used in this dataset will have all three fields, i.e., the datasets will look like someone called `dropna` on them to eliminate null values in those columns. I acknowledge that this means that what is left may not be representative of the raw-text citations that may be out there in the wild; however, as mentioned in the above paragraph, those three fields are fairly commonplace.

It is worth noting, however, that other fields are included in case we decide to incorporate them into our analysis later. They are less common and less reliable, so I would only include them as a very late micro-optimization to our model, if at all.
* pages: The pages in which the work appeared in the container (book, anthology, journal issue, etc.) in which it was published, represented as a string containing digits, a hyphen, and then more digits (or just digits, if the work appears on only one page)
* volume: The volume in which the work appears, represented as a string.
* source_title: The name of the journal or other entity responsible for the publication of the work.
* issue: The issue in which the work appeared.

Matching surnames, matching titles, and matching years should be sufficient grounds for declaring a match. `simple_preprocess` from Gensim.utils with deacc=True should be enough to get reliable matches, if the fields can just be extracted correctly.



In [ ]:
# The 46K-row OpenCitations dataset will be loaded here.

## TestRunner

As before, I define a TestRunner class. For now at least, I am choosing to simply stick with percent accuracy in predicting year, author, and title, because the frequency with which those attributes give successful matches with the reported metadata should be a good predictor of the frequency with which they give successful matches between different texts in a corpus.

Logic for k-fold CVs or LOOCVs is not managed by this class. This class is strictly for computing and reporting metrics on a given dataset.

In [ ]:
class TestRunner:
  """Encapsulates logic for calculating model performance statistics."""
  def __init__(self, model, test_set):
    """Initializes the TestRunner with a trained MODEL that is Testable and a
    TEST_SET, which is a DataFrame that has the columns specified in the above
    section, "Loading Datasets."
    """
    self.model = model
    self.test_set = test_set
    # self.predictions will be a DataFrame of the same format as TEST_SET.
    # However, it is set to None here because it will be computed lazily.
    self.predictions = None
  def get_predictions(self):
    """Returns a DataFrame with the same columns as the DataFrame on which the
    model was trained, as described in the "Datasets" section of this notebook.
    """
    if self.predictions is None:
      self.predictions = self.model.predict(self.test_set.raw_text)
    return self.predictions
  def year_accuracy(self):
    """Returns the proportion of publication years that the model predicts
    accurately.
    """
    return np.mean(self.test_set.year == self.get_predictions().year)
  def surname_accuracy(self):
    """Returns the proportion of citations from which the set of all authors'
    surnames is predicted with perfect accuracy (within the simplifications of
    de-accenting and capitalization/punctuation removal).
    """
    return np.mean([
        (
            set(simple_preprocess(name.surname) for name in true_names)
            == set(simple_preprocess(name.surname) for name in predicted_names)
        ) for true_names, predicted_names
        in zip(self.test_set.author, self.get_predictions().author)
    ])
  def primary_author_surname_accuracy(self):
    """Returns the proportion of citations from which the primary author's
    surname is predicted correctly. The first author who is listed in the
    dataset will be interpreted as the primary author.
    """
    return np.mean([
        true_names[0].surname == predicted_names[0].surname
        for true_names, predicted_names
        in zip(self.test_set.author, self.get_predictions().author)
    ])
  def title_accuracy(self):
    """Returns the proportion of citations from which the title of the cited work
    is predicted with perfect accuracy (within the simplifications of
    de-accenting and capitalization/punctuation removal).
    """
    return np.mean(
        simple_preprocess(true_title) == simple_preprocess(predicted_title)
        for true_title, predicted_title
        in zip(self.test_set.title, self.get_predictions().title)
    )
  def quick_report(self):
    """Prints a report of model performance without confidence intervals.
    Intended for development (debugging, preliminary results, etc.) and not for
    final reporting.
    """
    t0 = time.time()
    get_predictions()
    elapsed_time = time.time() - t0
    print('Generated labels for {0:,} records in {:.4f} seconds ({:.4f} seconds'
          ' per record)'.format(
              len(self.test_set.index),
              elapsed_time,
              elapsed_time / len(self.test_set.index)))
    print('Surname accuracy (complete set): {:.4f}'.format(
        self.surname_accuracy()))
    print('Year accuracy: {:.4f}'.format(self.year_accuracy()))
    print('Title accuracy (with preproc): {:.4f}').format(self.title_accuracy())

This demonstrates the power of `simple_preprocess`, a simple, widely used utility whose function should be reasonably easy to explain to other people.

In [11]:
simple_preprocess('The cát, in the hát.', deacc=True) == simple_preprocess('the cAt  iN tHe haT!!!?', deacc=True)

True

In [ ]:
class CrossValidator:
  """Reports cross validation results for a given model and dataset."""
  # This should have a has-a relationship with TestRunner.
  # You know, it's possible sklearn has this. I will take a look.